In [1]:
import sys
import os
from cobra.io import read_sbml_model

sys.path.append(os.path.abspath("../../src"))

model = read_sbml_model("/Users/azddza/Electromics-project/models/gap_but_models/added_butyrate_exchange_reactions/MAG029_gapfilled_noO2.xml")

print(f"模型中包含 {len(model.reactions)} 个反应，{len(model.metabolites)} 个代谢物，{len(model.genes)} 个基因。")


Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
模型中包含 1006 个反应，949 个代谢物，688 个基因。


In [2]:
print(model.metabolites.get_by_id("cpd00211_e0").reactions)

frozenset({<Reaction TR_cpd00211_c0 at 0x15ccde1a0>, <Reaction EX_cpd00211_e0 at 0x15ccde0b0>})


In [2]:
for rxn in model.reactions:
    if "cpd00211" in rxn.id:
        print(rxn.id, rxn.reaction)

EX_cpd00211_e0 cpd00211_e0 <=> 
TR_cpd00211_c0 cpd00211_e0 <=> cpd00211_c0


In [3]:
import networkx as nx
G_model = nx.DiGraph()
for reaction in model.reactions:
    reactants = [met.id for met in reaction.reactants]
    products = [met.id for met in reaction.products]
    for r in reactants:
        for p in products:
            G_model.add_edge(r, p, reaction=reaction.id)

In [5]:
print("前10个代谢物节点:")
for i, node in enumerate(G_model.nodes):
    print(node)
    if i > 10: break

print("\n前10条反应边（从代谢物A → B）:")
for i, (u, v, data) in enumerate(G_model.edges(data=True)):
    print(f"{u} → {v} via reaction: {data.get('reaction')}")
    if i > 10: break


前10个代谢物节点:
cpd00130_e0
cpd00130_c0
cpd00971_c0
cpd00971_e0
cpd00051_c0
cpd00011_c0
cpd00152_c0
cpd00067_c0
cpd00002_c0
cpd00003_c0
cpd00012_c0
cpd00355_c0

前10条反应边（从代谢物A → B）:
cpd00130_e0 → cpd00130_c0 via reaction: rxn05605_c0
cpd00130_e0 → cpd00971_c0 via reaction: rxn05207_c0
cpd00130_e0 → cpd00067_c0 via reaction: rxn05605_c0
cpd00130_c0 → cpd00005_c0 via reaction: rxn00161_c0
cpd00130_c0 → cpd00011_c0 via reaction: rxn00161_c0
cpd00130_c0 → cpd00020_c0 via reaction: rxn00161_c0
cpd00130_c0 → cpd00001_c0 via reaction: rxn00799_c0
cpd00130_c0 → cpd00106_c0 via reaction: rxn00799_c0
cpd00971_e0 → cpd00130_c0 via reaction: rxn05207_c0
cpd00971_e0 → cpd00971_c0 via reaction: rxn09165_c0
cpd00971_e0 → cpd00129_c0 via reaction: rxn05221_c0
cpd00971_e0 → cpd00067_e0 via reaction: rxn05209_c0


In [4]:
import networkx as nx

# 读取社区网络图（代谢物为节点）
G = nx.read_graphml("cpd00211_community.graphml")  # 你生成的社区网络
print(f"图节点数: {G.number_of_nodes()}, 边数: {G.number_of_edges()}")


图节点数: 20151, 边数: 147336


In [5]:
target_met = "cpd00211_c0"  # 名称或 ID，需与图中的节点一致
if target_met not in G:
    print("目标产物在社区图中不存在")

# 获取与其直接或间接连通的代谢物（可看作社区）
from networkx.algorithms.components import connected_components

# 对无向图求子图（假设是无向的）
G_u = G.to_undirected()
components = list(nx.connected_components(G_u))

# 找包含目标物的子图
target_component = next((c for c in components if target_met in c), None)
G_sub = G.subgraph(target_component).copy()


In [6]:
# 从社区图中取出所有代谢物名称（需要确保命名方式一致，如带_c）
community_mets = set(G_sub.nodes)

# 在模型图中找这些代谢物是否能从某些模型已有代谢物可达
exchange_mets = set()
for rxn in model.exchanges:
    for met in rxn.metabolites:
        exchange_mets.add(met.id)

# 在模型图中，从每个 exchange 代谢物出发，检查能否到目标社区
valid_starts = []
for start in exchange_mets:
    for target in community_mets:
        try:
            if nx.has_path(G_model, start, target):
                valid_starts.append(start)
                break
        except:
            continue


In [7]:
print("推荐用于 gapfilling 的起点代谢物（在模型中存在且连通社区）:")
for met in valid_starts:
    print(met)


推荐用于 gapfilling 的起点代谢物（在模型中存在且连通社区）:
cpd00254_e0
cpd00205_e0
cpd00166_e0
cpd00322_e0
cpd00104_e0
cpd10515_e0
cpd00034_e0
cpd00066_e0
cpd00149_e0
cpd00058_e0
cpd00029_e0


In [13]:
import pickle
reaction_dict_path = "reaction_dict.pkl"

# ===== 加载 reaction_dict =====
with open(reaction_dict_path, "rb") as f:
    reaction_dict = pickle.load(f)


In [ ]:
##！！组合爆炸！！
from cobra.util.solver import linear_reaction_coefficients
from cobra import Reaction, Metabolite
from cobra.util import solver
from networkx.algorithms.simple_paths import shortest_simple_paths

target_met = "cpd00211_c0"  # 目标代谢物

results = []

for start in valid_starts[:1]:
    try:
        paths = list(nx.all_simple_paths(G, source=start, target=target_met, cutoff=3))
    except:
        continue

    for path in paths:
        rxn_ids = []
        missing_reactions = []
        
        # 遍历路径，获取反应 ID
        for i in range(len(path) - 1):
            try:
                rxn_id = G[path[i]][path[i+1]]["reaction"]
                rxn_ids.append(rxn_id)
            except:
                continue
        
        # 对应的 reaction ID 与模型对比
        existing_rxns = set(r.id for r in model.reactions)
        to_add_rxns = [rxn for rxn in rxn_ids if rxn not in existing_rxns]

        # 模拟添加到模型
        with model.copy() as m:
            for rxn_id in to_add_rxns:
                # 这里需预先准备 reaction_dict (反应ID → Reaction对象)
                new_rxn = reaction_dict[rxn_id].copy()
                m.add_reactions([new_rxn])
            
            # 添加 exchange for starting metabolite
            ex_rxns = [rxn for rxn in m.reactions if start in rxn.id]
            if ex_rxns:
                ex_rxns[0].lower_bound = -10.0
            else:
                # 若无 exchange，构建一个
                ex = Reaction(f"EX_{start}")
                ex.name = f"Exchange for {start}"
                ex.lower_bound = -10
                ex.upper_bound = 1000
                ex.add_metabolites({m.metabolites.get_by_id(start): -1})
                m.add_reactions([ex])

            # 设置目标通量
            m.objective = "EX_cpd00211_e0"

            sol = m.optimize()
            results.append({
                "start": start,
                "path": path,
                "missing_reactions": to_add_rxns,
                "num_missing": len(to_add_rxns),
                "flux": sol.objective_value if sol.status == "optimal" else 0
            })


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpt0hvv35y.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpy5tvkuyp.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp70_uramu.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmprswpnqyy.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpw0xekrbw.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp9uqykyju.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model f

Ignoring reaction 'rxn02879' since it already exists.


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp47_vdt6l.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmppreu9feq.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpkibyt6el.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpcjvxejqu.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpzqczmgsr.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros


Ignoring reaction 'rxn38354' since it already exists.


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpc4jvjvug.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpp32zqc1l.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpwh9p2lhj.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp0w_iuu1j.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros


...

Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp4ysp47wi.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp_izunrbj.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpifvlfk1c.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmps2axqgnm.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp50wf7mes.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros


Ignoring reaction 'rxn38354' since it already exists.


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpkny8_b5s.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmptc00c4um.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpi5i5z2yy.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp1ibutzew.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros


Ignoring reaction 'rxn45696' since it already exists.


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpdjgful2d.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpqe5rxow3.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp862cjn48.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpaeiivma2.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpz6tklk5j.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpo70qmg61.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Read LP format model f

In [68]:
from cobra.util.solver import linear_reaction_coefficients
from cobra import Reaction, Metabolite
from cobra.util import solver
from networkx.algorithms.simple_paths import shortest_simple_paths
from cobra.flux_analysis import find_blocked_reactions

# 自定义 dead-end 检查函数
def find_dead_ends_manual(model):
    dead_ends = []
    for met in model.metabolites:
        producing = any(rxn.metabolites[met] > 0 for rxn in met.reactions)
        consuming = any(rxn.metabolites[met] < 0 for rxn in met.reactions)
        if not (producing and consuming):
            dead_ends.append(met.id)
    return dead_ends

target_met = "cpd00211_c0"  # 目标代谢物

results = []

for start in valid_starts[:1]:  
    try:
        paths = list(nx.all_simple_paths(G, source=start, target=target_met, cutoff=3))
    except:
        continue

    for path in paths:
        rxn_ids = []
        missing_reactions = []

        # 获取路径中的反应 ID
        for i in range(len(path) - 1):
            try:
                rxn_id = G[path[i]][path[i + 1]]["reaction"]
                rxn_ids.append(rxn_id)
            except:
                continue

        existing_rxns = set(r.id for r in model.reactions)
        to_add_rxns = [rxn for rxn in rxn_ids if rxn not in existing_rxns]

        with model.copy() as m:
            for rxn_id in to_add_rxns:
                if rxn_id in reaction_dict:
                    new_rxn = reaction_dict[rxn_id].copy()
                    m.add_reactions([new_rxn])

            # 添加起点代谢物的 exchange
            ex_id = f"EX_{start}"
            if ex_id in m.reactions:
                m.reactions.get_by_id(ex_id).lower_bound = -10
            else:
                try:
                    ex = Reaction(ex_id)
                    ex.name = f"Exchange for {start}"
                    ex.lower_bound = -10
                    ex.upper_bound = 1000
                    ex.add_metabolites({m.metabolites.get_by_id(start): -1})
                    m.add_reactions([ex])
                except KeyError:
                    print(f" 起点代谢物 {start} 不存在于模型中")
                    continue

            # 添加目标产物的 exchange reaction
            target_ex_id = "EX_cpd00211_e0"
            if target_ex_id not in m.reactions:
                try:
                    met = m.metabolites.get_by_id("cpd00211_e0")
                    target_ex = Reaction(target_ex_id)
                    target_ex.name = "Exchange for cpd00211_e0"
                    target_ex.lower_bound = 0
                    target_ex.upper_bound = 1000
                    target_ex.add_metabolites({met: -1})
                    m.add_reactions([target_ex])
                except KeyError:
                    print(" 目标代谢物 cpd00211_e0 不存在")
                    continue

            m.objective = target_ex_id

            sol = m.optimize()
            flux = sol.objective_value if sol.status == "optimal" else 0

            #  添加 blocked reaction 和 dead-end metabolite 检查
            blocked_rxns = set(find_blocked_reactions(m))
            dead_ends = set(find_dead_ends_manual(m))

            rxns_in_path = set(rxn_ids)
            mets_in_path = set()
            for rxn_id in rxn_ids:
                if rxn_id in m.reactions:
                    mets_in_path.update(m.id for m in m.reactions.get_by_id(rxn_id).metabolites)

            blocked_in_path = list(rxns_in_path & blocked_rxns)
            dead_ends_in_path = list(mets_in_path & dead_ends)

            results.append({
                "start": start,
                "path": path,
                "missing_reactions": to_add_rxns,
                "num_missing": len(to_add_rxns),
                "flux": flux,
                "blocked_reactions": blocked_in_path,
                "dead_end_metabolites": dead_ends_in_path
            })


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpcszgyt_u.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpi6apxd35.lp
Reading time = 0.00 seconds
: 954 rows, 2019 columns, 9075 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpgxwsfts3.lp
Reading time = 0.00 seconds
: 954 rows, 2019 columns, 9075 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp57kztc7l.lp
Reading time = 0.01 

Ignoring reaction 'rxn02879' since it already exists.


Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpxrhhl7ov.lp
Reading time = 0.00 seconds
: 952 rows, 2017 columns, 9061 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp63vhze79.lp
Reading time = 0.00 seconds
: 952 rows, 2017 columns, 9061 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpd8zrt9ff.lp
Reading time = 0.00 seconds
: 952 rows, 2017 columns, 9061 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - ex

Ignoring reaction 'rxn38354' since it already exists.


Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp2na4plus.lp
Reading time = 0.00 seconds
: 952 rows, 2017 columns, 9061 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpm97ifxxh.lp
Reading time = 0.00 seconds
: 952 rows, 2017 columns, 9061 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpmuxu74lv.lp
Reading time = 0.00 seconds
: 952 rows, 2017 columns, 9061 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - ex

Ignoring reaction 'rxn45696' since it already exists.


Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmphth_8jce.lp
Reading time = 0.00 seconds
: 952 rows, 2017 columns, 9061 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp6h103_vd.lp
Reading time = 0.00 seconds
: 952 rows, 2017 columns, 9061 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpqq8t04vb.lp
Reading time = 0.00 seconds
: 952 rows, 2017 columns, 9061 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - ex

In [69]:
results

[{'start': 'cpd00254_e0',
  'path': ['cpd00254_e0', 'cpd00067_c0', 'cpd00001_c0', 'cpd00211_c0'],
  'missing_reactions': ['rxn10571', 'rxn48566', 'rxn45696'],
  'num_missing': 3,
  'flux': 0.0,
  'blocked_reactions': ['rxn45696', 'rxn48566'],
  'dead_end_metabolites': ['cpd32214_c0',
   'cpd00211_c0',
   'cpd17485_c0',
   'cpd24202_c0',
   'cpd35107_c0']},
 {'start': 'cpd00254_e0',
  'path': ['cpd00254_e0', 'cpd00067_c0', 'cpd00003_c0', 'cpd00211_c0'],
  'missing_reactions': ['rxn10571', 'rxn48555', 'rxn36395'],
  'num_missing': 3,
  'flux': 0.0,
  'blocked_reactions': ['rxn36395', 'rxn48555'],
  'dead_end_metabolites': ['cpd05490_c0',
   'cpd31860_c0',
   'cpd00211_c0',
   'cpd24799_c0']},
 {'start': 'cpd00254_e0',
  'path': ['cpd00254_e0', 'cpd00067_c0', 'cpd00004_c0', 'cpd00211_c0'],
  'missing_reactions': ['rxn10571', 'rxn47755', 'rxn47214'],
  'num_missing': 3,
  'flux': 0.0,
  'blocked_reactions': ['rxn47214', 'rxn47755'],
  'dead_end_metabolites': ['cpd01011_c0',
   'cpd27749_c0

In [70]:
all_zero = all(entry['flux'] == 0 for entry in results)
print(f" 是否所有 flux 都为 0: {all_zero}")

 是否所有 flux 都为 0: True


In [27]:
from cobra import Reaction, Metabolite
import networkx as nx
from cobra.util.solver import linear_reaction_coefficients
target_met = "cpd00211_c0"

results = []

for start in valid_starts:  

    print(f"\n▶️ 起点: {start}")

    # Step 1: 找到 shortest path
    try:
        path = nx.shortest_path(G, source=start, target=target_met)
        print(f"找到路径: {path}")
    except nx.NetworkXNoPath:
        print(" 无路径可达")
        continue
    except Exception as e:
        print(f" 搜索出错: {e}")
        continue

    # Step 2: 获取路径中所有 reaction ID
    rxn_ids = []
    for i in range(len(path) - 1):
        try:
            rxn_id = G[path[i]][path[i+1]]["reaction"]
            rxn_ids.append(rxn_id)
        except:
            print(f"边 {path[i]} -> {path[i+1]} 缺失 reaction 属性")
            continue

    print(f"路径反应 IDs: {rxn_ids}")

    # Step 3: 模型中缺失的反应
    existing_rxns = set(r.id for r in model.reactions)
    to_add_rxns = [rxn for rxn in rxn_ids if rxn not in existing_rxns]
    print(f"缺失反应: {to_add_rxns}")

    # Step 4: 在模型副本中尝试添加反应并优化
    with model.copy() as m:
        for rxn_id in to_add_rxns:
            if rxn_id in reaction_dict:
                new_rxn = reaction_dict[rxn_id].copy()
                m.add_reactions([new_rxn])
            else:
                print(f" reaction_dict 缺失反应 {rxn_id}")

        # 添加或启用 exchange reaction for start
        ex_id = f"EX_{start}"
        if ex_id in m.reactions:
            m.reactions.get_by_id(ex_id).lower_bound = -10
        else:
            try:
                ex = Reaction(ex_id)
                ex.name = f"Exchange for {start}"
                ex.lower_bound = -10
                ex.upper_bound = 1000
                ex.add_metabolites({m.metabolites.get_by_id(start): -1})
                m.add_reactions([ex])
                print(f" 添加 exchange 反应: {ex_id}")
            except KeyError:
                print(f" 模型中找不到代谢物: {start}")
                continue

        # 设置目标产物 exchange 为目标
        target_rxn_id = "EX_cpd00211_e0"
        #target_rxn_id = f"EX_{target_met}"
        if target_rxn_id in m.reactions:
            m.objective = m.reactions.get_by_id(target_rxn_id)
        else:
            print(f" 模型中缺失目标 exchange: {target_rxn_id}")
            continue

        # 优化模型
        sol = m.optimize()
        flux = sol.objective_value if sol.status == "optimal" else 0
        print(f"✅ 通量: {flux}")

        # 记录结果
        results.append({
            "start": start,
            "path": path,
            "missing_reactions": to_add_rxns,
            "num_missing": len(to_add_rxns),
            "flux": flux
        })



▶️ 起点: cpd00254_e0
找到路径: ['cpd00254_e0', 'cpd00008_c0', 'cpd00211_c0']
路径反应 IDs: ['rxn10571', 'rxn01236']
缺失反应: ['rxn10571', 'rxn01236']
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpy7q4ixbz.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
✅ 通量: 0.0

▶️ 起点: cpd00205_e0
找到路径: ['cpd00205_e0', 'cpd00008_c0', 'cpd00211_c0']
路径反应 IDs: ['rxn18910', 'rxn01236']
缺失反应: ['rxn18910', 'rxn01236']
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpms7qkbjx.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
✅ 通量: 0.0

▶️ 起点: cpd00166_e0
找到路径: ['cpd00166_e0', 'cpd00008_c0', 'cpd00211_c0']
路径反应 IDs: ['rxn18671', 'rxn01236']
缺失反应: ['rxn18671', 'rxn01236']
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp3_wg6iox.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
✅ 通量: 0.0

▶️ 起点: cpd00322_e0
找到路径: ['cpd00322_e0', 'cpd00008_c0', 'cpd00211_

In [26]:
results

[]

In [28]:
all_zero = all(entry['flux'] == 0 for entry in results)
print(f" 是否所有 flux 都为 0: {all_zero}")

 是否所有 flux 都为 0: True


In [ ]:
# ✅ 自动添加关键 cofactor
from cobra import Reaction, Metabolite
import networkx as nx
from cobra.util.solver import linear_reaction_coefficients

target_met = "cpd00211_e0"  # 目标丁酸产物
results = []

for start in valid_starts:
    print(f"\n▶️ 起点: {start}")

    # Step 1: 找到最短路径
    try:
        path = nx.shortest_path(G, source=start, target=target_met)
        print(f"📍 路径: {path}")
    except nx.NetworkXNoPath:
        print("❌ 无路径可达")
        continue
    except Exception as e:
        print(f"❌ 搜索异常: {e}")
        continue

    # Step 2: 提取路径中的 reaction IDs
    rxn_ids = []
    for i in range(len(path) - 1):
        try:
            rxn_id = G[path[i]][path[i+1]]["reaction"]
            rxn_ids.append(rxn_id)
        except:
            print(f"⚠️ 路径边 {path[i]} → {path[i+1]} 缺失 reaction")
            continue

    print(f"🧬 路径反应 IDs: {rxn_ids}")

    # Step 3: 模型中缺失的反应
    existing_rxns = set(r.id for r in model.reactions)
    to_add_rxns = [rxn for rxn in rxn_ids if rxn not in existing_rxns]
    print(f"🛠️ 缺失反应: {to_add_rxns}")

    # Step 4: 模型副本中模拟添加
    with model.copy() as m:
        # 添加缺失反应
        for rxn_id in to_add_rxns:
            if rxn_id in reaction_dict:
                new_rxn = reaction_dict[rxn_id].copy()
                m.add_reactions([new_rxn])
                print(f"✅ 添加反应: {rxn_id}")
            else:
                print(f"❌ reaction_dict 缺失反应: {rxn_id}")

        # ✅ 自动添加关键 cofactor 的 demand（如无 exchange）
        cofactors = {
            "cpd00010_c0": "DM_cpd00010_c0",  # CoA
            "cpd00002_c0": "DM_cpd00002_c0",  # ATP
            "cpd00004_c0": "DM_cpd00004_c0",  # NADH
        }

        for met_id, rxn_id in cofactors.items():
            try:
                met = m.metabolites.get_by_id(met_id)
                if rxn_id not in m.reactions:
                    demand = Reaction(rxn_id)
                    demand.name = f"Demand for {met_id}"
                    demand.lower_bound = 0
                    demand.upper_bound = 1000
                    demand.add_metabolites({met: -1})
                    m.add_reactions([demand])
                    print(f"✅ 添加 cofactor demand: {rxn_id}")
            except KeyError:
                print(f"⚠️ 模型中找不到 cofactor: {met_id}")

        # 添加或启用起点 exchange
        ex_id = f"EX_{start}"
        if ex_id in m.reactions:
            m.reactions.get_by_id(ex_id).lower_bound = -10
        else:
            try:
                ex = Reaction(ex_id)
                ex.name = f"Exchange for {start}"
                ex.lower_bound = -10
                ex.upper_bound = 1000
                ex.add_metabolites({m.metabolites.get_by_id(start): -1})
                m.add_reactions([ex])
                print(f"✅ 添加起点 exchange: {ex_id}")
            except KeyError:
                print(f"❌ 模型中缺失代谢物: {start}")
                continue

        # 设置目标 exchange 为目标函数
        target_rxn_id = f"EX_{target_met}"
        if target_rxn_id in m.reactions:
            m.objective = m.reactions.get_by_id(target_rxn_id)
        else:
            try:
                target_met_obj = m.metabolites.get_by_id(target_met)
                target_ex = Reaction(target_rxn_id)
                target_ex.name = f"Exchange for {target_met}"
                target_ex.lower_bound = 0
                target_ex.upper_bound = 1000
                target_ex.add_metabolites({target_met_obj: -1})
                m.add_reactions([target_ex])
                m.objective = target_ex
                print(f"✅ 自动添加目标 exchange: {target_rxn_id}")
            except:
                print(f"❌ 添加目标 exchange 失败: {target_rxn_id}")
                continue

        # Step 5: 优化
        sol = m.optimize()
        flux = sol.objective_value if sol.status == "optimal" else 0
        print(f"🚀 通量结果: {flux}")

        # 记录结果
        results.append({
            "start": start,
            "path": path,
            "missing_reactions": to_add_rxns,
            "num_missing": len(to_add_rxns),
            "flux": flux
        })




▶️ 起点: cpd00254_e0
📍 路径: ['cpd00254_e0', 'cpd00067_c0', 'cpd00211_e0']
🧬 路径反应 IDs: ['rxn10571', 'rxn08183']
🛠️ 缺失反应: ['rxn10571', 'rxn08183']
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpqhrg_z_r.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros
✅ 添加反应: rxn10571
✅ 添加反应: rxn08183
✅ 添加 cofactor demand: DM_cpd00010_c0
✅ 添加 cofactor demand: DM_cpd00002_c0
✅ 添加 cofactor demand: DM_cpd00004_c0
🚀 通量结果: 0.0

▶️ 起点: cpd00205_e0
📍 路径: ['cpd00205_e0', 'cpd00067_c0', 'cpd00211_e0']
🧬 路径反应 IDs: ['rxn38724', 'rxn08183']
🛠️ 缺失反应: ['rxn38724', 'rxn08183']
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpfsaue6w0.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
✅ 添加反应: rxn38724
✅ 添加反应: rxn08183
✅ 添加 cofactor demand: DM_cpd00010_c0
✅ 添加 cofactor demand: DM_cpd00002_c0
✅ 添加 cofactor demand: DM_cpd00004_c0
🚀 通量结果: 0.0

▶️ 起点: cpd00166_e0
📍 路径: ['cpd00166_e0', 'cpd00067_c0', 'cpd00211_e0']
🧬 

In [ ]:
all_zero = all(entry['flux'] == 0 for entry in results)
print(f" 是否所有 flux 都为 0: {all_zero}")

📢 是否所有 flux 都为 0: True


In [ ]:
rxn = reaction_dict.get("rxn01236")
rxn.reaction


'cpd00002_c0 + cpd00211_c0 <=> cpd00008_c0 + cpd01662_c0'

In [72]:
###最短路径 gapfilling

from cobra import Reaction, Metabolite
import networkx as nx
from cobra.util.solver import linear_reaction_coefficients
from cobra.flux_analysis import find_blocked_reactions

# 自定义 dead-end 检查函数
def find_dead_ends_manual(model):
    dead_ends = []
    for met in model.metabolites:
        producing = any(rxn.metabolites[met] > 0 for rxn in met.reactions)
        consuming = any(rxn.metabolites[met] < 0 for rxn in met.reactions)
        if not (producing and consuming):
            dead_ends.append(met.id)
    return dead_ends

target_met = "cpd00211_c0"
results = []

for start in valid_starts:  
    
    print(f"\n▶️ 起点: {start}")
    try:
        path = nx.shortest_path(G, source=start, target=target_met)
        print(f" 路径: {path}")
    except Exception as e:
        print(f" 搜索异常: {e}")
        continue

    rxn_ids = []
    for i in range(len(path) - 1):
        try:
            rxn_id = G[path[i]][path[i + 1]]["reaction"]
            rxn_ids.append(rxn_id)
        except:
            print(f" 边 {path[i]} → {path[i+1]} 缺失 reaction")
            continue

    print(f"🧬 路径反应 IDs: {rxn_ids}")

    existing_rxns = set(r.id for r in model.reactions)
    to_add_rxns = [rxn for rxn in rxn_ids if rxn not in existing_rxns]
    print(f" 缺失反应: {to_add_rxns}")

    with model.copy() as m:
        for rxn_id in to_add_rxns:
            if rxn_id in reaction_dict:
                new_rxn = reaction_dict[rxn_id].copy()
                m.add_reactions([new_rxn])
                print(f" 添加反应: {rxn_id}")
            else:
                print(f" reaction_dict 缺失反应: {rxn_id}")

        ex_id = f"EX_{start}"
        if ex_id in m.reactions:
            m.reactions.get_by_id(ex_id).lower_bound = -10
        else:
            try:
                ex = Reaction(ex_id)
                ex.name = f"Exchange for {start}"
                ex.lower_bound = -10
                ex.upper_bound = 1000
                ex.add_metabolites({m.metabolites.get_by_id(start): -1})
                m.add_reactions([ex])
                print(f" 添加 exchange: {ex_id}")
            except KeyError:
                print(f" 找不到代谢物: {start}")
                continue

        target_rxn_id = "EX_cpd00211_e0"
        if target_rxn_id in m.reactions:
            m.objective = m.reactions.get_by_id(target_rxn_id)
        else:
            try:
                target_met_obj = m.metabolites.get_by_id("cpd00211_e0")
                target_ex = Reaction(target_rxn_id)
                target_ex.name = f"Exchange for cpd00211_e0"
                target_ex.lower_bound = 0
                target_ex.upper_bound = 1000
                target_ex.add_metabolites({target_met_obj: -1})
                m.add_reactions([target_ex])
                m.objective = target_ex
                print(f"添加目标 exchange: {target_rxn_id}")
            except KeyError:
                print(f" 无法添加目标 exchange，缺少代谢物: cpd00211_e0")
                continue

        sol = m.optimize()
        flux = sol.objective_value if sol.status == "optimal" else 0
        print(f" 通量结果: {flux}")

        blocked_rxns = set(find_blocked_reactions(m))
        blocked_in_path = [r for r in rxn_ids if r in blocked_rxns]
        if blocked_in_path:
            print(f" 路径中有被阻断的反应: {blocked_in_path}")

        dead_mets = set(find_dead_ends_manual(m))
        mets_in_path = set()
        for rxn_id in rxn_ids:
            rxn = m.reactions.get_by_id(rxn_id)
            mets_in_path.update(m.id for m in rxn.metabolites)
        dead_in_path = [m for m in mets_in_path if m in dead_mets]
        if dead_in_path:
            print(f" 路径中存在 dead-end 代谢物: {dead_in_path}")

        print(" 各反应通量:")
        for rxn_id in rxn_ids:
            try:
                flux_val = sol.fluxes.get(rxn_id, 0)
                print(f"  - {rxn_id}: {flux_val}")
            except:
                print(f"  - {rxn_id}: 不在模型中")

        results.append({
            "start": start,
            "path": path,
            "missing_reactions": to_add_rxns,
            "num_missing": len(to_add_rxns),
            "flux": flux,
            "blocked_reactions": blocked_in_path,
            "dead_end_metabolites": dead_in_path,
        })



▶️ 起点: cpd00254_e0
 路径: ['cpd00254_e0', 'cpd00008_c0', 'cpd00211_c0']
🧬 路径反应 IDs: ['rxn10571', 'rxn01236']
 缺失反应: ['rxn10571', 'rxn01236']
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp40485_jp.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 添加反应: rxn10571
 添加反应: rxn01236
 通量结果: 0.0
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpbuct36vb.lp
Reading time = 0.00 seconds
: 951 rows, 2017 columns, 9059 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp2j9yx6xs.lp
Reading time = 0.00 seconds
: 951 rows, 2017 columns, 9059 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970


In [73]:
results

[{'start': 'cpd00254_e0',
  'path': ['cpd00254_e0', 'cpd00008_c0', 'cpd00211_c0'],
  'missing_reactions': ['rxn10571', 'rxn01236'],
  'num_missing': 2,
  'flux': 0.0,
  'blocked_reactions': ['rxn01236'],
  'dead_end_metabolites': ['cpd01662_c0']},
 {'start': 'cpd00205_e0',
  'path': ['cpd00205_e0', 'cpd00008_c0', 'cpd00211_c0'],
  'missing_reactions': ['rxn18910', 'rxn01236'],
  'num_missing': 2,
  'flux': 0.0,
  'blocked_reactions': ['rxn01236'],
  'dead_end_metabolites': ['cpd00205_e0', 'cpd01662_c0']},
 {'start': 'cpd00166_e0',
  'path': ['cpd00166_e0', 'cpd00008_c0', 'cpd00211_c0'],
  'missing_reactions': ['rxn18671', 'rxn01236'],
  'num_missing': 2,
  'flux': 0.0,
  'blocked_reactions': ['rxn01236'],
  'dead_end_metabolites': ['cpd00166_e0', 'cpd01662_c0']},
 {'start': 'cpd00322_e0',
  'path': ['cpd00322_e0', 'cpd00008_c0', 'cpd00211_c0'],
  'missing_reactions': ['rxn08745', 'rxn01236'],
  'num_missing': 2,
  'flux': 0.0,
  'blocked_reactions': ['rxn01236'],
  'dead_end_metabolite

In [74]:
import pandas as pd

df = pd.DataFrame(results)
df.to_csv("gapfill_results.csv", index=False)



In [77]:

record = {
    "start": start,
    "path": path,
    "missing_reactions": to_add_rxns,
    "num_missing": len(to_add_rxns),
    "flux": flux,
    "blocked_reactions": blocked_in_path,
    "dead_end_metabolites": dead_ends_in_path
}

# ✅ 尝试修复 dead-end 并重新计算 flux
extra_rxns = set()
for rxn_id, rxn in reaction_dict.items():
    for met, coeff in rxn.metabolites.items():
        if met.id in dead_ends_in_path and coeff < 0:  # 尝试消费 dead-end
            extra_rxns.add(rxn_id)
            break

rescued_flux = 0.0
if extra_rxns:
    with model.copy() as m2:
        # 添加原缺失反应
        for rxn_id in to_add_rxns:
            if rxn_id in reaction_dict:
                m2.add_reactions([reaction_dict[rxn_id].copy()])
        # 添加补救反应
        for rxn_id in extra_rxns:
            if rxn_id in reaction_dict and rxn_id not in m2.reactions:
                m2.add_reactions([reaction_dict[rxn_id].copy()])
        # exchange 和目标设置（复制原逻辑）
        ex_id = f"EX_{start}"
        if ex_id not in m2.reactions:
            try:
                ex = Reaction(ex_id)
                ex.lower_bound = -10
                ex.upper_bound = 1000
                ex.add_metabolites({m2.metabolites.get_by_id(start): -1})
                m2.add_reactions([ex])
            except:
                pass
        if "EX_cpd00211_e0" not in m2.reactions:
            try:
                met = m2.metabolites.get_by_id("cpd00211_e0")
                ex2 = Reaction("EX_cpd00211_e0")
                ex2.lower_bound = 0
                ex2.upper_bound = 1000
                ex2.add_metabolites({met: -1})
                m2.add_reactions([ex2])
            except:
                pass
        m2.objective = m2.reactions.get_by_id("EX_cpd00211_e0")
        sol2 = m2.optimize()
        rescued_flux = sol2.objective_value if sol2.status == "optimal" else 0.0

record["rescued_reactions"] = list(extra_rxns)
record["rescued_count"] = len(extra_rxns)
record["rescued_flux"] = rescued_flux

results.append(record)



Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpcuuxv_t4.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros


In [79]:
print(f"尝试补救代谢物: {dead_ends_in_path}")
print(f"添加补救反应数: {len(extra_rxns)}")
print(f"补救后通量: {rescued_flux}")


尝试补救代谢物: ['cpd07946_c0', 'cpd00211_c0', 'cpd00120_c0', 'cpd00728_c0']
添加补救反应数: 57
补救后通量: 1000.0


In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df

,start,path,missing_reactions,num_missing,flux,blocked_reactions,dead_end_metabolites,rescued_reactions,rescued_count,rescued_flux
0,cpd00254_e0,"[cpd00254_e0, cpd00008_c0, cpd00211_c0]","[rxn10571, rxn01236]",2,0.0,[rxn01236],[cpd01662_c0],NaN,NaN,NaN
1,cpd00205_e0,"[cpd00205_e0, cpd00008_c0, cpd00211_c0]","[rxn18910, rxn01236]",2,0.0,[rxn01236],"[cpd00205_e0, cpd01662_c0]",NaN,NaN,NaN
2,cpd00166_e0,"[cpd00166_e0, cpd00008_c0, cpd00211_c0]","[rxn18671, rxn01236]",2,0.0,[rxn01236],"[cpd00166_e0, cpd01662_c0]",NaN,NaN,NaN
3,cpd00322_e0,"[cpd00322_e0, cpd00008_c0, cpd00211_c0]","[rxn08745, rxn01236]",2,0.0,[rxn01236],"[cpd00322_e0, cpd01662_c0]",NaN,NaN,NaN
4,cpd00104_e0,"[cpd00104_e0, cpd00008_c0, cpd00211_c0]","[rxn05223, rxn01236]",2,0.0,[rxn01236],"[cpd00104_e0, cpd01662_c0]",NaN,NaN,NaN
5,cpd10515_e0,"[cpd10515_e0, cpd00008_c0, cpd00211_c0]","[rxn08472, rxn01236]",2,0.0,[rxn01236],[cpd01662_c0],NaN,NaN,NaN
6,cpd00034_e0,"[cpd00034_e0, cpd00008_c0, cpd00211_c0]","[rxn09392, rxn01236]",2,0.0,[rxn01236],"[cpd00034_e0, cpd01662_c0]",NaN,NaN,NaN
7,cpd00066_e0,"[cpd00066_e0, cpd00008_c0, cpd00211_c0]","[rxn18584, rxn01236]",2,0.0,[rxn01236],"[cpd01662_c0, cpd00066_e0]",NaN,NaN,NaN
8,cpd00149_e0,"[cpd00149_e0, cpd00008_c0, cpd00211_c0]","[rxn05166, rxn01236]",2,0.0,[rxn01236],[cpd01662_c0],NaN,NaN,NaN
9,cpd00058_e0,"[cpd00058_e0, cpd00001_c0, cpd00211_c0]","[rxn29072, rxn45696]",2,0.0,[rxn45696],"[cpd32214_c0, cpd00211_c0, cpd24202_c0]",NaN,NaN,NaN


In [81]:
# 显示成功补救通量的路径
df[df["rescued_flux"] > 0][["start", "rescued_flux", "rescued_count", "rescued_reactions"]]


,start,rescued_flux,rescued_count,rescued_reactions
11,cpd00029_e0,1000.0,57.0,"[rxn08184, rxn35683, rxn46479, rxn37165, rxn00..."
12,cpd00029_e0,1000.0,57.0,"[rxn08184, rxn35683, rxn46479, rxn37165, rxn00..."


In [92]:
# 假设你已经有 df，其中包含列 rescued_reactions
rescue_rxns = df[df["rescued_flux"] > 0]["rescued_reactions"].iloc[0]  # 提取第一行的反应列表
core_path_rxns = ["rxn31359", "rxn13427"]

minimal_set = set(rescue_rxns)
baseline_flux = 1000.0

for rxn in rescue_rxns:
    test_set = minimal_set - {rxn}

    with model.copy() as m:
        for r in core_path_rxns + list(test_set):
            if r in reaction_dict:
                m.add_reactions([reaction_dict[r].copy()])
        
        # 设置目标
        m.objective = m.reactions.get_by_id("EX_cpd00211_e0")
        m.reactions.get_by_id(f"EX_{start}").lower_bound = -10

        flux = m.optimize().objective_value
        if flux >= baseline_flux * 0.999:  # 允许一点浮动
            print(f" 移除 {rxn} 不影响 flux → 删除")
            minimal_set.remove(rxn)
        else:
            print(f" 移除 {rxn} 导致 flux 下降 → 保留")


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmphgcqle17.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros


Ignoring reaction 'rxn13427' since it already exists.


 移除 rxn08184 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpscdr8p9_.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros


Ignoring reaction 'rxn13427' since it already exists.


 移除 rxn35683 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp668ifcwi.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros


...

 移除 rxn16586 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpsqg11cnx.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros


Ignoring reaction 'rxn13427' since it already exists.


 移除 rxn43573 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmprnotss4s.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros


Ignoring reaction 'rxn13427' since it already exists.


 移除 rxn43619 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpc2de4360.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 移除 rxn13427 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmptax9wfxb.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 移除 rxn31413 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpu5koggxw.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 移除 rxn03642 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpvqujjgjx.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 移除 rxn00994 不影响 flux → 删除
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp0_vlrtls.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 移除 rxn31412 不影响 flux → 删除
Read LP format model fr

In [93]:
minimal_set

{'rxn00875', 'rxn10074'}

In [94]:
final_rxns_to_add = core_path_rxns + list(minimal_set)
print(f" 最终要添加的反应总数: {len(final_rxns_to_add)}")


 最终要添加的反应总数: 4


In [96]:
target_rxn_id = "EX_cpd00211_e0"
with model.copy() as m:
    for rxn_id in final_rxns_to_add:
        if rxn_id in reaction_dict:
            m.add_reactions([reaction_dict[rxn_id].copy()])
    m.objective = target_rxn_id
    sol = m.optimize()
    flux = sol.objective_value if sol.status == "optimal" else 0
    print(f"\n 添加最终反应集后的通量: {flux}")


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp3we0w8rz.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros

 添加最终反应集后的通量: 1000.0


In [ ]:
from cobra.flux_analysis import flux_variability_analysis
# ✅ 主路径反应列表（根据你自己模型设定）
core_path_rxns = ["rxn31359", "rxn13427"]  # 示例，替换为你起点→目标路径的反应

# ✅ 补救反应集（可以从 DataFrame 中提取）
rescue_rxns = list(df[df["rescued_flux"] > 0]["rescued_reactions"].iloc[0])

with model.copy() as m:
    for r in core_path_rxns + rescue_rxns:
        if r in reaction_dict:
            m.add_reactions([reaction_dict[r].copy()])

    fva = flux_variability_analysis(m, reaction_list=rescue_rxns)

    essential_rxns = fva[(fva["maximum"].abs() > 1e-6) | (fva["minimum"].abs() > 1e-6)].index.tolist()
    print(f" 最小必需反应集大小: {len(essential_rxns)}")


Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpesrfr49k.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros


Ignoring reaction 'rxn13427' since it already exists.


Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpem3fk5hg.lp
Reading time = 0.00 seconds
: 978 rows, 2129 columns, 9563 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpcalsuv5a.lp
Reading time = 0.00 seconds
: 978 rows, 2129 columns, 9563 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - expires 2026-05-12
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpgfgfjrvm.lp
Reading time = 0.00 seconds
: 978 rows, 2129 columns, 9563 nonzeros
Set parameter Username
Set parameter LicenseID to value 2663970
Academic license - for non-commercial use only - ex

In [84]:
enhanced_results = []

for res in results:
    start = res["start"]
    base_rxns = res["missing_reactions"]
    dead_mets = res["dead_end_metabolites"]

    for dead_met in dead_mets:
        try:
            # 反向图中从 dead_met → start 找最短路径
            reverse_path = nx.shortest_path(G.reverse(), source=dead_met, target=start)
            print(f"dead-end {dead_met} → 起点 {start} 路径: {reverse_path}")
        except:
            continue

        # 获取修复路径上的反应
        repair_rxns = []
        for i in range(len(reverse_path) - 1):
            try:
                rxn_id = G[reverse_path[i]][reverse_path[i + 1]]["reaction"]
                repair_rxns.append(rxn_id)
            except:
                continue

        # 合并所有需添加的反应
        all_rxns_to_add = list(set(base_rxns + repair_rxns))

        with model.copy() as m:
            for rxn_id in all_rxns_to_add:
                if rxn_id in reaction_dict:
                    new_rxn = reaction_dict[rxn_id].copy()
                    m.add_reactions([new_rxn])

            # 添加起点 exchange
            ex_id = f"EX_{start}"
            if ex_id not in m.reactions:
                try:
                    ex = Reaction(ex_id)
                    ex.lower_bound = -10
                    ex.upper_bound = 1000
                    ex.add_metabolites({m.metabolites.get_by_id(start): -1})
                    m.add_reactions([ex])
                except:
                    continue
            else:
                m.reactions.get_by_id(ex_id).lower_bound = -10

            # 添加目标产物 exchange
            target_ex_id = "EX_cpd00211_e0"
            if target_ex_id not in m.reactions:
                try:
                    target_met = m.metabolites.get_by_id("cpd00211_e0")
                    ex = Reaction(target_ex_id)
                    ex.lower_bound = 0
                    ex.upper_bound = 1000
                    ex.add_metabolites({target_met: -1})
                    m.add_reactions([ex])
                except:
                    continue
            m.objective = target_ex_id

            # 优化
            sol = m.optimize()
            flux = sol.objective_value if sol.status == "optimal" else 0
            print(f" 尝试修复 dead-end {dead_met} 后通量: {flux}")

            enhanced_results.append({
                "start": start,
                "original_dead_end": dead_met,
                "original_missing": base_rxns,
                "repair_rxns": repair_rxns,
                "flux_after_repair": flux
            })


dead-end cpd01662_c0 → 起点 cpd00254_e0 路径: ['cpd01662_c0', 'cpd00009_c0', 'cpd00254_e0']
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmpm4sc39dk.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 尝试修复 dead-end cpd01662_c0 后通量: 0.0
dead-end cpd00205_e0 → 起点 cpd00205_e0 路径: ['cpd00205_e0']
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp07fyvdax.lp
Reading time = 0.00 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 尝试修复 dead-end cpd00205_e0 后通量: 0.0
dead-end cpd01662_c0 → 起点 cpd00205_e0 路径: ['cpd01662_c0', 'cpd00009_c0', 'cpd00205_e0']
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp0zhnosml.lp
Reading time = 0.01 seconds
: 949 rows, 2012 columns, 9034 nonzeros
 尝试修复 dead-end cpd01662_c0 后通量: 0.0
dead-end cpd00166_e0 → 起点 cpd00166_e0 路径: ['cpd00166_e0']
Read LP format model from file /var/folders/6w/knrbtrj125ggkrx091kd2g840000gn/T/tmp0bbz5fg7.lp
Reading time = 0

In [87]:
import pandas as pd

dff = pd.DataFrame(enhanced_results)
dff


,start,original_dead_end,original_missing,repair_rxns,flux_after_repair
0,cpd00254_e0,cpd01662_c0,"[rxn10571, rxn01236]",[rxn00871],0.0
1,cpd00205_e0,cpd00205_e0,"[rxn18910, rxn01236]",[],0.0
2,cpd00205_e0,cpd01662_c0,"[rxn18910, rxn01236]","[rxn00871, rxn13842]",0.0
3,cpd00166_e0,cpd00166_e0,"[rxn18671, rxn01236]",[],0.0
4,cpd00166_e0,cpd01662_c0,"[rxn18671, rxn01236]",[rxn00871],0.0
5,cpd00322_e0,cpd00322_e0,"[rxn08745, rxn01236]",[],0.0
6,cpd00322_e0,cpd01662_c0,"[rxn08745, rxn01236]",[rxn00871],0.0
7,cpd00104_e0,cpd00104_e0,"[rxn05223, rxn01236]",[],0.0
8,cpd00104_e0,cpd01662_c0,"[rxn05223, rxn01236]","[rxn00871, rxn05223]",0.0
9,cpd10515_e0,cpd01662_c0,"[rxn08472, rxn01236]",[rxn00871],0.0
